In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('C:/Football Analytics/Data/Premier League 2021-22/MUNEVE.csv', index_col = 0)
df = df[df['event'] == 'Pass']
df = df[df['outcome'] == 'Successful']
df

,player,kit,team,x,y,endx,endy,event,outcome,time
0,Allan,6,31,49.9,50.0,31.9,51.8,Pass,Successful,0
1,Yerry Mina,13,31,31.9,51.8,31.8,55.7,Pass,Successful,0
3,David de Gea,1,32,3.0,46.4,10.8,83.7,Pass,Successful,0
4,Raphaël Varane,19,32,21.6,82.8,27.3,95.4,Pass,Successful,0
5,Luke Shaw,23,32,27.3,95.3,18.4,82.1,Pass,Successful,0
...,...,...,...,...,...,...,...,...,...,...
1523,Abdoulaye Doucouré,16,31,34.7,5.5,35.8,3.5,Pass,Successful,92
1524,Andros Townsend,14,31,40.3,5.6,51.3,24.8,Pass,Successful,92
1530,Scott McTominay,39,32,51.7,47.0,65.6,49.0,Pass,Successful,92
1543,Tom Davies,26,31,67.3,15.1,72.5,9.7,Pass,Successful,94


In [3]:
#Will multiply x-coordinates by 1.2 and y-coordinates by 0.8 because the default Threat Grid provided
#Default Threat Grid from StatsBomb/Opta is configured for a 120*80 pitch

df['x'] = 1.2*df['x']
df['endx'] = 1.2*df['endx']
df['y'] = 0.8*df['y']
df['endy'] = 0.8*df['endy']

df

,player,kit,team,x,y,endx,endy,event,outcome,time
0,Allan,6,31,59.88,40.00,38.28,41.44,Pass,Successful,0
1,Yerry Mina,13,31,38.28,41.44,38.16,44.56,Pass,Successful,0
3,David de Gea,1,32,3.60,37.12,12.96,66.96,Pass,Successful,0
4,Raphaël Varane,19,32,25.92,66.24,32.76,76.32,Pass,Successful,0
5,Luke Shaw,23,32,32.76,76.24,22.08,65.68,Pass,Successful,0
...,...,...,...,...,...,...,...,...,...,...
1523,Abdoulaye Doucouré,16,31,41.64,4.40,42.96,2.80,Pass,Successful,92
1524,Andros Townsend,14,31,48.36,4.48,61.56,19.84,Pass,Successful,92
1530,Scott McTominay,39,32,62.04,37.60,78.72,39.20,Pass,Successful,92
1543,Tom Davies,26,31,80.76,12.08,87.00,7.76,Pass,Successful,94


In [4]:
#Uploading Threat Grid data (threat generated in each zone) and then combining it with passes

xT = pd.read_csv('C:/Football Analytics/Data/xT_Grid.csv')
xT = np.array(xT)
xT_rows, xT_cols = xT.shape

In [5]:
#Grouping starting and ending x-y coordinates of passes

df['x1_bin'] = pd.cut(df['x'], bins=xT_cols, labels=False)
df['y1_bin'] = pd.cut(df['y'], bins=xT_rows, labels=False)
df['x2_bin'] = pd.cut(df['endx'], bins=xT_cols, labels=False)
df['y2_bin'] = pd.cut(df['endy'], bins=xT_rows, labels=False)

In [6]:
#Calculating the threat generated by passes

df['start_zone_value'] = df[['x1_bin', 'y1_bin']].apply(lambda x: xT[x[1]][x[0]], axis=1)
df['end_zone_value'] = df[['x2_bin', 'y2_bin']].apply(lambda x: xT[x[1]][x[0]], axis=1)
df['xT'] = df['end_zone_value'] - df['start_zone_value']

df

,player,kit,team,x,y,endx,endy,event,outcome,time,x1_bin,y1_bin,x2_bin,y2_bin,start_zone_value,end_zone_value,xT
0,Allan,6,31,59.88,40.00,38.28,41.44,Pass,Successful,0,5,3,3,3,0.014846,0.011324,-0.003522
1,Yerry Mina,13,31,38.28,41.44,38.16,44.56,Pass,Successful,0,3,3,3,3,0.011324,0.011324,0.000000
3,David de Gea,1,32,3.60,37.12,12.96,66.96,Pass,Successful,0,0,3,0,5,0.009411,0.007501,-0.001910
4,Raphaël Varane,19,32,25.92,66.24,32.76,76.32,Pass,Successful,0,2,5,3,6,0.009424,0.009777,0.000353
5,Luke Shaw,23,32,32.76,76.24,22.08,65.68,Pass,Successful,0,3,6,1,5,0.009777,0.008786,-0.000991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1523,Abdoulaye Doucouré,16,31,41.64,4.40,42.96,2.80,Pass,Successful,92,4,0,4,0,0.012147,0.012147,0.000000
1524,Andros Townsend,14,31,48.36,4.48,61.56,19.84,Pass,Successful,92,4,0,6,1,0.012147,0.016856,0.004709
1530,Scott McTominay,39,32,62.04,37.60,78.72,39.20,Pass,Successful,92,6,3,7,3,0.016895,0.019971,0.003075
1543,Tom Davies,26,31,80.76,12.08,87.00,7.76,Pass,Successful,94,8,1,8,0,0.024122,0.024015,-0.000107


In [ ]:
#df.to_csv('C:/Football Analytics/Data/MUNEVE - xThreat.csv')